In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

### Request for Census Tract Level Population data from Census API

In [3]:
stateCode = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16',
             '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
            '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 
             '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']

In [4]:
len(stateCode)

51

In [5]:
import requests
import pandas as pd
from datetime import datetime
import time
import json
from io import StringIO

In [6]:
def collectAll_county(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs3?get=NAME,'+ variables + '&for=county:*&in=state:*&key=250f58e35283d400e4185b197e3abe9021079afe'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

#### Total population: B01003_001E

In [7]:
year = '2007'
variables = 'B01003_001E'

In [8]:
pop_09 = collectAll_county(year,variables)
pop_09.head()

['NAME', 'B01003_001E', 'state', 'county']


,county_name,B01003_001E,state code,county code
0,"Leake County, Mississippi",22636,28,079
1,"Lee County, Mississippi",79343,28,081
2,"Leflore County, Mississippi",35229,28,083
3,"Lincoln County, Mississippi",34169,28,085
4,"Lowndes County, Mississippi",59559,28,087


In [9]:
pop_09.shape

(1882, 4)

#### 1.per capita income: B19301_001E

In [10]:
variables = 'B19301_001E'
df1 = collectAll_county(year,variables)

['NAME', 'B19301_001E', 'state', 'county']


In [11]:
df1.shape

(1882, 4)

In [12]:
df1 = df1.rename(columns = {'B19301_001E': 'per_capita_income'})
df1['fips'] = df1['state code'] + df1['county code']
df1.head()

,county_name,per_capita_income,state code,county code,fips
0,"Mille Lacs County, Minnesota",21236,27,095,27095
1,"Morrison County, Minnesota",22348,27,097,27097
2,"Mower County, Minnesota",23956,27,099,27099
3,"Nicollet County, Minnesota",24917,27,103,27103
4,"Nobles County, Minnesota",21047,27,105,27105


#### 2.Percent of Population Below Poverty Level: (C17002_002E+C17002_003E)/C17002_001E

In [13]:
variables = 'C17002_002E,C17002_003E,C17002_001E'
df2 = collectAll_county(year,variables)

['NAME', 'C17002_002E', 'C17002_003E', 'C17002_001E', 'state', 'county']


In [14]:
df2['C17002_002E'] = df2['C17002_002E'].astype('int')
df2['C17002_003E'] = df2['C17002_003E'].astype('int')
df2['C17002_001E'] = df2['C17002_001E'].astype('int')

In [15]:
df2['percent_below_poverty'] = ((df2['C17002_002E'] + df2['C17002_003E'])/df2['C17002_001E'])*100

In [16]:
df2['fips'] = df2['state code'] + df2['county code']
df2.head()

,county_name,C17002_002E,C17002_003E,C17002_001E,state code,county code,percent_below_poverty,fips
0,"Anderson County, Kentucky",499,1137,20396,21,005,8.021181,21005
1,"Barren County, Kentucky",3985,4011,40080,21,009,19.950100,21009
2,"Bell County, Kentucky",3619,5523,28789,21,013,31.755184,21013
3,"Boone County, Kentucky",3213,4251,107585,21,015,6.937770,21015
4,"Boyd County, Kentucky",3032,4337,45461,21,019,16.209498,21019


In [17]:
df2.shape

(1882, 8)

#### 3.Percent of Population 25+ with Less than a 12th Grade Education: (B15002_003E,B15002_004E,B15002_005E,B15002_003E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E)/B15003_001E

In [18]:
def collectAll_county2(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs1?get=NAME,'+ variables + '&for=county:*&in=state:*'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

In [19]:
variables = 'B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E'
df3 = collectAll_county(year,variables)

['NAME', 'B15002_003E', 'B15002_004E', 'B15002_005E', 'B15002_006E', 'B15002_007E', 'B15002_008E', 'B15002_009E', 'B15002_010E', 'B15002_020E', 'B15002_021E', 'B15002_022E', 'B15002_023E', 'B15002_024E', 'B15002_025E', 'B15002_026E', 'B15002_027E', 'B15002_001E', 'state', 'county']


In [20]:
df3.shape

(1882, 20)

In [21]:
def changeInteger(value):
    if value != None:
        return int(value)
    else:
        return np.nan

In [22]:
df3['B15002_003E'] = df3['B15002_003E'].apply(changeInteger)
df3['B15002_004E'] = df3['B15002_004E'].apply(changeInteger)
df3['B15002_005E'] = df3['B15002_005E'].apply(changeInteger)
df3['B15002_006E'] = df3['B15002_006E'].apply(changeInteger)
df3['B15002_007E'] = df3['B15002_007E'].apply(changeInteger)
df3['B15002_008E'] = df3['B15002_008E'].apply(changeInteger)
df3['B15002_009E'] = df3['B15002_009E'].apply(changeInteger)
df3['B15002_010E'] = df3['B15002_010E'].apply(changeInteger)
df3['B15002_020E'] = df3['B15002_020E'].apply(changeInteger)
df3['B15002_021E'] = df3['B15002_021E'].apply(changeInteger)
df3['B15002_022E'] = df3['B15002_022E'].apply(changeInteger)
df3['B15002_023E'] = df3['B15002_023E'].apply(changeInteger)
df3['B15002_024E'] = df3['B15002_024E'].apply(changeInteger)
df3['B15002_025E'] = df3['B15002_025E'].apply(changeInteger)
df3['B15002_026E'] = df3['B15002_026E'].apply(changeInteger)
df3['B15002_027E'] = df3['B15002_027E'].apply(changeInteger)
df3['B15002_001E'] = df3['B15002_001E'].apply(changeInteger)

In [23]:
df3['percent_no_highschool_diploma'] = ((df3['B15002_003E']+df3['B15002_004E']+df3['B15002_005E']+df3['B15002_006E']+df3['B15002_007E']+df3['B15002_008E']+df3['B15002_009E']
                                       +df3['B15002_010E']+df3['B15002_020E']+df3['B15002_021E']+df3['B15002_022E']+df3['B15002_023E']+df3['B15002_024E']+df3['B15002_025E']+df3['B15002_003E']
                                       +df3['B15002_026E']+df3['B15002_027E'])/df3['B15002_001E'])*100

In [24]:
df3['fips'] = df3['state code'] + df3['county code']
df3.head()

,county_name,B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,...,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E,state code,county code,percent_no_highschool_diploma,fips
0,"Cumberland County, Maine",332.0,216.0,298.0,1340.0,1014.0,1070.0,887.0,1500.0,729.0,...,1682.0,1030.0,1239.0,1056.0,1519.0,190501.0,23,005,7.853502,23005
1,"Franklin County, Maine",55.0,0.0,31.0,287.0,199.0,210.0,347.0,260.0,15.0,...,257.0,198.0,164.0,277.0,297.0,20237.0,23,007,13.213421,23007
2,"Hancock County, Maine",27.0,36.0,30.0,380.0,243.0,434.0,442.0,345.0,20.0,...,268.0,223.0,421.0,283.0,116.0,38165.0,23,009,8.780296,23009
3,"Kennebec County, Maine",72.0,70.0,101.0,1248.0,738.0,920.0,637.0,557.0,62.0,...,1114.0,620.0,564.0,775.0,614.0,84176.0,23,011,10.189365,23011
4,"Knox County, Maine",0.0,57.0,38.0,720.0,380.0,197.0,364.0,208.0,28.0,...,347.0,204.0,423.0,127.0,204.0,29793.0,23,013,11.066358,23013


#### 4.Percent of Population Living in Mobile Homes: (B25033_006E+B25033_007E+B25033_012E+B25033_013E)/B25033_001E

In [25]:
variables = 'B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E'
df4 = collectAll_county(year,variables)

['NAME', 'B25033_006E', 'B25033_007E', 'B25033_012E', 'B25033_013E', 'B25033_001E', 'state', 'county']


In [26]:
df4.shape

(1882, 8)

In [27]:
df4['B25033_006E'] = df4['B25033_006E'].apply(changeInteger)
df4['B25033_007E'] = df4['B25033_007E'].apply(changeInteger)
df4['B25033_012E'] = df4['B25033_012E'].apply(changeInteger)
df4['B25033_013E'] = df4['B25033_013E'].apply(changeInteger)
df4['B25033_001E'] = df4['B25033_001E'].apply(changeInteger)

In [28]:
df4['percent_mobile_homes'] = ((df4['B25033_006E'] + df4['B25033_007E'] + df4['B25033_012E'] + df4['B25033_013E'])/df4['B25033_001E'])*100

In [29]:
df4['fips'] = df4['state code'] + df4['county code']
df4.head()

,county_name,B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E,state code,county code,percent_mobile_homes,fips
0,"Plumas County, California",1876.0,0.0,1263.0,0.0,20793.0,06,063,15.096427,06063
1,"Riverside County, California",110584.0,3426.0,40059.0,1086.0,1973219.0,06,065,7.863040,06065
2,"Sacramento County, California",23619.0,1175.0,4250.0,76.0,1346758.0,06,067,2.162230,06067
3,"San Benito County, California",781.0,14.0,709.0,0.0,50391.0,06,069,2.984660,06069
4,"San Bernardino County, California",68907.0,890.0,17467.0,667.0,1938512.0,06,071,4.536005,06071


#### 5.Percent of Population with No Vehicle Available: (B25044_003E+B25044_010E)/B25044_001E

In [30]:
variables = 'B25044_003E,B25044_010E,B25044_001E'
df5 = collectAll_county(year,variables)

['NAME', 'B25044_003E', 'B25044_010E', 'B25044_001E', 'state', 'county']


In [31]:
df5.shape

(1882, 6)

In [32]:
df5['B25044_003E'] = df5['B25044_003E'].apply(changeInteger)
df5['B25044_010E'] = df5['B25044_010E'].apply(changeInteger)
df5['B25044_001E'] = df5['B25044_001E'].apply(changeInteger)

In [33]:
df5['percent_no_vehicle'] = ((df5['B25044_003E'] + df5['B25044_010E'])/df5['B25044_001E'])*100

In [34]:
df5['fips'] = df5['state code'] + df5['county code']
df5.head()

,county_name,B25044_003E,B25044_010E,B25044_001E,state code,county code,percent_no_vehicle,fips
0,"Franklin County, Alabama",247,438,12192,01,059,5.618438,01059
1,"Geneva County, Alabama",208,246,10308,01,061,4.404346,01061
2,"Houston County, Alabama",737,1744,36562,01,069,6.785734,01069
3,"Jackson County, Alabama",473,837,21129,01,071,6.200009,01071
4,"Jefferson County, Alabama",6607,14575,266554,01,073,7.946607,01073


#### 6.Percent of Population Unemployed:sum(B21005_006E, B21005_007E, B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E)/B21005_001E

In [35]:
variables = 'B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E'
df6 = collectAll_county2(year,variables)

['NAME', 'B21005_006E', 'B21005_007E', 'B21005_011E', 'B21005_012E', 'B21005_017E', 'B21005_022E', 'B21005_028E', 'B21005_029E', 'B21005_033E', 'B21005_034E', 'B21005_001E', 'state', 'county']


In [36]:
df6.shape

(800, 14)

In [37]:
df6['B21005_006E'] = df6['B21005_006E'].apply(changeInteger)
df6['B21005_007E'] = df6['B21005_007E'].apply(changeInteger)
df6['B21005_011E'] = df6['B21005_011E'].apply(changeInteger)
df6['B21005_012E'] = df6['B21005_012E'].apply(changeInteger)
df6['B21005_017E'] = df6['B21005_017E'].apply(changeInteger)
df6['B21005_022E'] = df6['B21005_022E'].apply(changeInteger)
df6['B21005_028E'] = df6['B21005_028E'].apply(changeInteger)
df6['B21005_029E'] = df6['B21005_029E'].apply(changeInteger)
df6['B21005_033E'] = df6['B21005_033E'].apply(changeInteger)
df6['B21005_034E'] = df6['B21005_034E'].apply(changeInteger)
df6['B21005_001E'] = df6['B21005_001E'].apply(changeInteger)

In [38]:
df6['percent_unemployed_CDC'] = ((df6['B21005_006E']+df6['B21005_007E']+df6['B21005_011E']+df6['B21005_012E']+df6['B21005_017E']+df6['B21005_022E']+df6['B21005_028E']+df6['B21005_029E']+df6['B21005_033E']+df6['B21005_034E'])/df6['B21005_001E'])*100

In [39]:
df6['fips'] = df6['state code'] + df6['county code']
df6.head()

,county_name,B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E,state code,county code,percent_unemployed_CDC,fips
0,"Grayson County, Texas",0,314,1243,5400,135,1232,0,1405,0,4372,71876,48,181,19.618510,48181
1,"Gregg County, Texas",0,38,1229,6425,72,1082,59,907,441,4357,70820,48,183,20.629766,48183
2,"Guadalupe County, Texas",0,92,981,5684,0,315,0,769,161,3980,69507,48,187,17.238552,48187
3,"Harris County, Texas",921,2355,63521,229063,1764,35936,1828,14791,9737,109927,2495172,48,201,18.830085,48201
4,"Hays County, Texas",471,131,3215,15664,44,768,18,808,68,3031,96174,48,209,25.181442,48209


#### 7.Percent of Population Living In Accommodations with Less Than 1 Room Per Person/Crowding: (B25014_005E+B25014_006E+B25014_007E+B25014_011E+B25014_012E+B25014_013E)/B25014_001E

In [40]:
variables = 'B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E'
df7 = collectAll_county(year,variables)

['NAME', 'B25014_005E', 'B25014_006E', 'B25014_007E', 'B25014_011E', 'B25014_012E', 'B25014_013E', 'B25014_001E', 'state', 'county']


In [41]:
df7.shape

(1882, 10)

In [42]:
df7['B25014_005E'] = df7['B25014_005E'].apply(changeInteger)
df7['B25014_006E'] = df7['B25014_006E'].apply(changeInteger)
df7['B25014_007E'] = df7['B25014_007E'].apply(changeInteger)
df7['B25014_011E'] = df7['B25014_011E'].apply(changeInteger)
df7['B25014_012E'] = df7['B25014_012E'].apply(changeInteger)
df7['B25014_013E'] = df7['B25014_013E'].apply(changeInteger)
df7['B25014_001E'] = df7['B25014_001E'].apply(changeInteger)

In [43]:
df7['percent_overcrowding'] = ((df7['B25014_005E']+df7['B25014_006E']+df7['B25014_007E']+df7['B25014_011E']+df7['B25014_012E']+df7['B25014_013E'])/df7['B25014_001E'])*100

In [44]:
df7['fips'] = df7['state code'] + df7['county code']
df7.head()

,county_name,B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E,state code,county code,percent_overcrowding,fips
0,"District of Columbia, District of Columbia",996.0,183.0,25.0,4253.0,2487.0,478.0,249805.0,11,001,3.371430,11001
1,"Alachua County, Florida",335.0,62.0,0.0,987.0,180.0,30.0,95850.0,12,001,1.663015,12001
2,"Bay County, Florida",477.0,160.0,0.0,994.0,337.0,32.0,71417.0,12,005,2.800454,12005
3,"Brevard County, Florida",1180.0,213.0,40.0,1081.0,292.0,0.0,217708.0,12,009,1.288882,12009
4,"Broward County, Florida",6201.0,954.0,87.0,9012.0,2172.0,464.0,676384.0,12,011,2.792792,12011


#### 8.Percent of Housing Units with 10+ Units in Structure: (B25024_007E+B25024_008E+B25024_009E)/B25024_001E

In [45]:
variables = 'B25024_007E,B25024_008E,B25024_009E,B25024_001E'
df8 = collectAll_county(year,variables)

['NAME', 'B25024_007E', 'B25024_008E', 'B25024_009E', 'B25024_001E', 'state', 'county']


In [46]:
df8.shape

(1882, 7)

In [47]:
df8['B25024_007E'] = df8['B25024_007E'].apply(changeInteger)
df8['B25024_008E'] = df8['B25024_008E'].apply(changeInteger)
df8['B25024_009E'] = df8['B25024_009E'].apply(changeInteger)
df8['B25024_001E'] = df8['B25024_001E'].apply(changeInteger)

In [48]:
df8['percent_multi_unit_housing'] = ((df8['B25024_007E']+df8['B25024_008E']+df8['B25024_009E'])/df8['B25024_001E'])*100

In [49]:
df8['fips'] = df8['state code'] + df8['county code']
df8.head()

,county_name,B25024_007E,B25024_008E,B25024_009E,B25024_001E,state code,county code,percent_multi_unit_housing,fips
0,"Richmond County, Georgia",2358.0,967.0,1659.0,86325.0,13,245,5.773530,13245
1,"Rockdale County, Georgia",989.0,643.0,244.0,30209.0,13,247,6.210070,13247
2,"Spalding County, Georgia",364.0,176.0,318.0,25821.0,13,255,3.322877,13255
3,"Stephens County, Georgia",39.0,21.0,47.0,12274.0,13,257,0.871761,13257
4,"Sumter County, Georgia",76.0,69.0,11.0,14200.0,13,261,1.098592,13261


#### 9.Percent Of Population 65+: (B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_049E)/B01001_001E

In [50]:
variables = 'B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E'
df9 = collectAll_county(year,variables)

['NAME', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E', 'B01001_001E', 'state', 'county']


In [51]:
df9.shape

(1882, 16)

In [52]:
df9['B01001_020E'] = df9['B01001_020E'].apply(changeInteger)
df9['B01001_021E'] = df9['B01001_021E'].apply(changeInteger)
df9['B01001_022E'] = df9['B01001_022E'].apply(changeInteger)
df9['B01001_023E'] = df9['B01001_023E'].apply(changeInteger)
df9['B01001_024E'] = df9['B01001_024E'].apply(changeInteger)
df9['B01001_025E'] = df9['B01001_025E'].apply(changeInteger)
df9['B01001_044E'] = df9['B01001_044E'].apply(changeInteger)
df9['B01001_045E'] = df9['B01001_045E'].apply(changeInteger)
df9['B01001_046E'] = df9['B01001_046E'].apply(changeInteger)
df9['B01001_047E'] = df9['B01001_047E'].apply(changeInteger)
df9['B01001_048E'] = df9['B01001_048E'].apply(changeInteger)
df9['B01001_049E'] = df9['B01001_049E'].apply(changeInteger)
df9['B01001_001E'] = df9['B01001_001E'].apply(changeInteger)

In [53]:
df9['percent_age_65_and_older'] = ((df9['B01001_020E']+df9['B01001_021E']+df9['B01001_022E']+df9['B01001_023E']+df9['B01001_024E']+df9['B01001_025E']+df9['B01001_044E']
                                  +df9['B01001_045E']+df9['B01001_046E']+df9['B01001_047E']+df9['B01001_048E']+df9['B01001_049E'])/df9['B01001_001E'])*100

In [54]:
df9['fips'] = df9['state code'] + df9['county code']
df9.head()

,county_name,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E,state code,county code,percent_age_65_and_older,fips
0,"Cumberland County, Maine",2022,2383,4016,3177,2032,1700,2149,2946,4552,4722,3577,3753,274712,23,005,13.479207,23005
1,"Franklin County, Maine",251,302,535,568,173,112,219,357,588,512,301,359,29840,23,007,14.333110,23007
2,"Hancock County, Maine",486,674,889,764,456,494,464,725,1086,783,607,1092,53208,23,009,16.012630,23009
3,"Kennebec County, Maine",940,1324,1862,1528,1076,629,962,1698,2051,2192,1770,1355,120559,23,011,14.421984,23011
4,"Knox County, Maine",386,619,576,721,328,320,391,576,925,933,900,389,40879,23,013,17.280266,23013


#### 10.Percent Of Population Who Do Not Speak English: B99163_005E/B99163_001E

In [55]:
variables = 'B99163_005E,B99163_001E'
df10 = collectAll_county(year,variables)

['NAME', 'B99163_005E', 'B99163_001E', 'state', 'county']


In [56]:
df10.shape

(1882, 5)

In [57]:
df10['B99163_005E'] = df10['B99163_005E'].apply(changeInteger)
df10['B99163_001E'] = df10['B99163_001E'].apply(changeInteger)

In [58]:
df10['percent_limited_english_abilities'] = (df10['B99163_005E']/df10['B99163_001E'])*100

In [59]:
df10['fips'] = df10['state code'] + df10['county code']
df10.head()

,county_name,B99163_005E,B99163_001E,state code,county code,percent_limited_english_abilities,fips
0,"Alcorn County, Mississippi",660,32969,28,003,2.001881,28003
1,"Bolivar County, Mississippi",680,34607,28,011,1.964920,28011
2,"Clay County, Mississippi",165,19573,28,025,0.842998,28025
3,"Coahoma County, Mississippi",238,25447,28,027,0.935277,28027
4,"Copiah County, Mississippi",595,26977,28,029,2.205583,28029


#### 11.Percent of Population 17 Years of Age and Under: (B01001_003E+B01001_004E+B01001_005E+B01001_006E+B01001_027E+B01001_028E+B01001_029E+B01001_030E)/B01003_001E

In [60]:
variables = 'B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E'
df11 = collectAll_county(year,variables)

['NAME', 'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E', 'B01003_001E', 'state', 'county']


In [61]:
df11.shape

(1882, 12)

In [62]:
df11['B01001_003E'] = df11['B01001_003E'].apply(changeInteger)
df11['B01001_004E'] = df11['B01001_004E'].apply(changeInteger)
df11['B01001_005E'] = df11['B01001_005E'].apply(changeInteger)
df11['B01001_006E'] = df11['B01001_006E'].apply(changeInteger)
df11['B01001_027E'] = df11['B01001_027E'].apply(changeInteger)
df11['B01001_028E'] = df11['B01001_028E'].apply(changeInteger)
df11['B01001_029E'] = df11['B01001_029E'].apply(changeInteger)
df11['B01001_030E'] = df11['B01001_030E'].apply(changeInteger)
df11['B01003_001E'] = df11['B01003_001E'].apply(changeInteger)

In [63]:
df11['percent_age_17_and_younger'] = ((df11['B01001_003E']+df11['B01001_004E']+df11['B01001_005E']+df11['B01001_006E']+df11['B01001_027E']+df11['B01001_028E']+df11['B01001_029E']+df11['B01001_030E'])/df11['B01003_001E'])*100

In [64]:
df11['fips'] = df11['state code'] + df11['county code']
df11.head()

,county_name,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E,state code,county code,percent_age_17_and_younger,fips
0,"Leake County, Mississippi",914,1107,881,505,834,878,627,799,22636,28,079,28.914119,28079
1,"Lee County, Mississippi",3196,3059,3089,1807,2991,2966,2746,1862,79343,28,081,27.369774,28081
2,"Leflore County, Mississippi",1599,1406,1298,809,1244,1270,1286,932,35229,28,083,27.942888,28083
3,"Lincoln County, Mississippi",1221,1354,1068,549,1138,1632,747,910,34169,28,085,25.224619,28085
4,"Lowndes County, Mississippi",2319,2251,2376,1490,2137,2044,2175,1389,59559,28,087,27.168018,28087


#### 12.Percent Minority: 1-(B03002_003E/B03002_001E)

In [65]:
variables = 'B03002_003E,B03002_001E'
df12 = collectAll_county(year,variables)

['NAME', 'B03002_003E', 'B03002_001E', 'state', 'county']


In [66]:
df12.shape

(1882, 5)

In [67]:
df12['B03002_003E'] = df12['B03002_003E'].apply(changeInteger)
df12['B03002_001E'] = df12['B03002_001E'].apply(changeInteger)

In [68]:
df12['percent_minorities'] = (1 - (df12['B03002_003E']/df12['B03002_001E']))*100

In [69]:
df12['fips'] = df12['state code'] + df12['county code']
df12.head()

,county_name,B03002_003E,B03002_001E,state code,county code,percent_minorities,fips
0,"Lee County, Illinois",31636.0,35349.0,17,103,10.503833,17103
1,"McHenry County, Illinois",263111.0,309448.0,17,111,14.974083,17111
2,"McLean County, Illinois",138434.0,162031.0,17,113,14.563263,17113
3,"Madison County, Illinois",233799.0,266142.0,17,119,12.152535,17119
4,"Ogle County, Illinois",48714.0,54543.0,17,141,10.686981,17141


#### 13.Percent of Population with a Disability: (B18101_004E+B18101_007E+B18101_010E+B18101_013E+B18101_016E+B18101_019E)/B18101_002E + (B18101_023E+B18101_026E+B18101_029E+B18101_032E+B18101_035E+B18101_038E)/B18101_021E

In [70]:
variables = 'B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E'
df13 = collectAll_county2(year,variables)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df13.shape

In [72]:
df13['B18101_004E'] = df13['B18101_004E'].apply(changeInteger)
df13['B18101_007E'] = df13['B18101_007E'].apply(changeInteger)
df13['B18101_010E'] = df13['B18101_010E'].apply(changeInteger)
df13['B18101_013E'] = df13['B18101_013E'].apply(changeInteger)
df13['B18101_016E'] = df13['B18101_016E'].apply(changeInteger)
df13['B18101_002E'] = df13['B18101_002E'].apply(changeInteger)
df13['B18101_023E'] = df13['B18101_023E'].apply(changeInteger)
df13['B18101_026E'] = df13['B18101_026E'].apply(changeInteger)
df13['B18101_029E'] = df13['B18101_029E'].apply(changeInteger)
df13['B18101_032E'] = df13['B18101_032E'].apply(changeInteger)
df13['B18101_035E'] = df13['B18101_035E'].apply(changeInteger)
df13['B18101_038E'] = df13['B18101_038E'].apply(changeInteger)
df13['B18101_021E'] = df13['B18101_021E'].apply(changeInteger)

In [73]:
df13['percent_disabled'] = ((df13['B18101_004E']+df13['B18101_007E']+df13['B18101_010E']+df13['B18101_013E']+df13['B18101_016E'])/df13['B18101_002E'] + (df13['B18101_023E']+df13['B18101_026E']+df13['B18101_029E']+df13['B18101_032E']+df13['B18101_035E']+df13['B18101_038E'])/df13['B18101_021E'])*100

In [74]:
df13['fips'] = df13['state code'] + df13['county code']
df13.head()

,county_name,B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E,state code,county code,percent_disabled,fips
0,"El Paso County, Texas",0,4649,3632,15570,5729,7714,347383,214,2309,3585,17446,7555,13989,383941,48,141,20.261173,48141
1,"Fort Bend County, Texas",148,2730,2473,8295,2182,3709,272988,0,1637,1615,7413,2793,4939,278447,48,157,12.405059,48157
2,"Galveston County, Texas",107,1802,980,6907,2046,2725,138863,0,968,1080,6778,2709,4987,145464,48,167,19.885966,48167
3,"Grayson County, Texas",0,542,364,3851,1912,1603,57965,0,853,574,4168,1230,2649,60448,48,181,27.178194,48181
4,"Gregg County, Texas",93,1371,1257,3394,1186,1236,56665,0,750,568,4653,1267,3063,61612,48,183,29.603642,48183


#### 14.Percent of Population Living in Group Quarters: B26001_001E/B01003_001E

In [71]:
variables = 'B26001_001E,B01003_001E'
df14 = collectAll_county(year,variables)

['NAME', 'B26001_001E', 'B01003_001E', 'state', 'county']


In [72]:
df14.shape

(1882, 5)

In [73]:
df14['B26001_001E'] = df14['B26001_001E'].apply(changeInteger)
df14['B01003_001E'] = df14['B01003_001E'].apply(changeInteger)

In [74]:
df14['percent_institutionalized_in_group_quarters'] = (df14['B26001_001E']/df14['B01003_001E'])*100

In [75]:
df14['fips'] = df14['state code'] + df14['county code']
df14.head()

,county_name,B26001_001E,B01003_001E,state code,county code,percent_institutionalized_in_group_quarters,fips
0,"Leake County, Mississippi",702.0,22636,28,079,3.101255,28079
1,"Lee County, Mississippi",1891.0,79343,28,081,2.383323,28081
2,"Leflore County, Mississippi",1616.0,35229,28,083,4.587130,28083
3,"Lincoln County, Mississippi",238.0,34169,28,085,0.696538,28085
4,"Lowndes County, Mississippi",2830.0,59559,28,087,4.751591,28087


#### 15.Percent of Children Living in Single Parent Households: (B09008_010E+B09008_011E+B09008_012E)/B09008_001E

In [76]:
variables = 'B09008_010E,B09008_011E,B09008_012E,B09008_001E'
df15 = collectAll_county2(year,variables)

['NAME', 'B09008_010E', 'B09008_011E', 'B09008_012E', 'B09008_001E', 'state', 'county']


In [77]:
df15.shape

(800, 7)

In [78]:
df15['B09008_010E'] = df15['B09008_010E'].apply(changeInteger)
df15['B09008_011E'] = df15['B09008_011E'].apply(changeInteger)
df15['B09008_012E'] = df15['B09008_012E'].apply(changeInteger)
df15['B09008_001E'] = df15['B09008_001E'].apply(changeInteger)

In [79]:
df15['percent_single_parent_households_CDC'] = ((df15['B09008_010E'] + df15['B09008_011E']+df15['B09008_012E'])/df15['B09008_001E'])*100

In [80]:
df15['fips'] = df15['state code'] + df15['county code']
df15.head()

,county_name,B09008_010E,B09008_011E,B09008_012E,B09008_001E,state code,county code,percent_single_parent_households_CDC,fips
0,"Livingston Parish, Louisiana",2151.0,5188.0,39.0,31723.0,22,063,23.257573,22063
1,"Orleans Parish, Louisiana",1015.0,23832.0,53.0,44088.0,22,071,56.477953,22071
2,"Ouachita Parish, Louisiana",1223.0,15329.0,163.0,38990.0,22,073,42.869967,22073
3,"Rapides Parish, Louisiana",992.0,10895.0,235.0,32765.0,22,079,36.996795,22079
4,"St. Landry Parish, Louisiana",572.0,9126.0,0.0,24647.0,22,097,39.347588,22097


### age 18 to 34
#### (B01001_007E+B01001_008E+B01001_009E+B01001_010E+B01001_011E+B01001_012E+B01001_031E+B01001_032E+B01001_033E+B01001_034E+B01001_035E+B01001_036E)/B01001_001E

In [81]:
variables = 'B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E'
df16 = collectAll_county(year,variables)

['NAME', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E', 'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E', 'B01001_001E', 'state', 'county']


In [82]:
df16['B01001_007E'] = df16['B01001_007E'].apply(changeInteger)
df16['B01001_008E'] = df16['B01001_008E'].apply(changeInteger)
df16['B01001_009E'] = df16['B01001_009E'].apply(changeInteger)
df16['B01001_010E'] = df16['B01001_010E'].apply(changeInteger)
df16['B01001_011E'] = df16['B01001_011E'].apply(changeInteger)
df16['B01001_012E'] = df16['B01001_012E'].apply(changeInteger)
df16['B01001_031E'] = df16['B01001_031E'].apply(changeInteger)
df16['B01001_032E'] = df16['B01001_032E'].apply(changeInteger)
df16['B01001_033E'] = df16['B01001_033E'].apply(changeInteger)
df16['B01001_034E'] = df16['B01001_034E'].apply(changeInteger)
df16['B01001_035E'] = df16['B01001_035E'].apply(changeInteger)
df16['B01001_036E'] = df16['B01001_036E'].apply(changeInteger)
df16['B01001_001E'] = df16['B01001_001E'].apply(changeInteger)

In [83]:
df16['percent_age_18_to_34'] = ((df16['B01001_007E'] + df16['B01001_008E'] +df16['B01001_009E'] +df16['B01001_010E'] +df16['B01001_011E']+df16['B01001_012E']+df16['B01001_031E']+df16['B01001_032E']+df16['B01001_033E']+df16['B01001_034E']+df16['B01001_035E']+df16['B01001_036E'])/df16['B01001_001E'])*100

In [84]:
df16['fips'] = df16['state code'] + df16['county code']
df16.head()

,county_name,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E,state code,county code,percent_age_18_to_34,fips
0,"Cumberland County, Maine",3890,1554,2202,4606,6713,8544,3558,1729,2029,4143,6893,8305,274712,23,005,19.717377,23005
1,"Franklin County, Maine",468,417,189,548,884,702,744,301,425,533,951,821,29840,23,007,23.401475,23007
2,"Hancock County, Maine",706,425,316,1039,1299,1373,573,300,317,780,1357,1478,53208,23,009,18.724628,23009
3,"Kennebec County, Maine",1690,884,699,1983,3554,3182,1601,1043,666,2098,3761,3359,120559,23,011,20.338589,23011
4,"Knox County, Maine",389,209,182,746,1111,1268,394,117,441,416,1059,1144,40879,23,013,18.288119,23013


In [85]:
df16.shape

(1882, 18)

### Age 35 to 44
#### (B01001_013E+B01001_014E+B01001_037E+B01001_038E)/B01001_001E

In [86]:
variables = 'B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E'
df17 = collectAll_county(year,variables)

['NAME', 'B01001_013E', 'B01001_014E', 'B01001_037E', 'B01001_038E', 'B01001_001E', 'state', 'county']


In [87]:
df17['B01001_013E'] = df17['B01001_013E'].apply(changeInteger)
df17['B01001_014E'] = df17['B01001_014E'].apply(changeInteger)
df17['B01001_037E'] = df17['B01001_037E'].apply(changeInteger)
df17['B01001_038E'] = df17['B01001_038E'].apply(changeInteger)
df17['B01001_001E'] = df17['B01001_001E'].apply(changeInteger)

In [88]:
df17['percent_age_35_to_44'] = ((df17['B01001_013E']+df17['B01001_014E'] +df17['B01001_037E']+df17['B01001_038E'])/df17['B01001_001E'])*100

In [89]:
df17['fips'] = df17['state code'] + df17['county code']
df17.head()

,county_name,B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E,state code,county code,percent_age_35_to_44,fips
0,"Cumberland County, Maine",9527,11739,10730,11718,274712,23,005,15.912665,23005
1,"Franklin County, Maine",954,1037,978,1026,29840,23,007,13.388070,23007
2,"Hancock County, Maine",1493,2159,1842,1989,53208,23,009,14.063675,23009
3,"Kennebec County, Maine",4139,4391,3991,4927,120559,23,011,14.472582,23011
4,"Knox County, Maine",1363,1604,1354,1369,40879,23,013,13.919127,23013


In [90]:
df17.shape

(1882, 10)

### age 45 to 54
#### (B01001_015E+B01001_016E+B01001_039E+B01001_040E)/B01001_001E

In [91]:
variables = 'B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E'
df18 = collectAll_county(year,variables)

['NAME', 'B01001_015E', 'B01001_016E', 'B01001_039E', 'B01001_040E', 'B01001_001E', 'state', 'county']


In [92]:
df18['B01001_015E'] = df18['B01001_015E'].apply(changeInteger)
df18['B01001_016E'] = df18['B01001_016E'].apply(changeInteger)
df18['B01001_039E'] = df18['B01001_039E'].apply(changeInteger)
df18['B01001_040E'] = df18['B01001_040E'].apply(changeInteger)
df18['B01001_001E'] = df18['B01001_001E'].apply(changeInteger)

In [93]:
df18['percent_age_45_to_54'] = ((df18['B01001_015E']+df18['B01001_016E'] +df18['B01001_039E']+df18['B01001_040E'])/df18['B01001_001E'])*100

In [94]:
df18['fips'] = df18['state code'] + df18['county code']
df18.head()

,county_name,B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E,state code,county code,percent_age_45_to_54,fips
0,"Cumberland County, Maine",11558,10621,12525,11317,274712,23,005,16.752453,23005
1,"Franklin County, Maine",1195,1135,1145,1197,29840,23,007,15.656836,23007
2,"Hancock County, Maine",2228,2161,2314,2345,53208,23,009,17.004962,23009
3,"Kennebec County, Maine",5032,4682,5581,4932,120559,23,011,16.777677,23011
4,"Knox County, Maine",1688,1801,1707,1724,40879,23,013,16.928007,23013


In [95]:
df18.shape

(1882, 10)

### age 55 to 64
#### (B01001_017E+B01001_018E+B01001_019E+B01001_041E+B01001_042E+B01001_043E)/B01001_001E

In [96]:
variables = 'B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E'
df19 = collectAll_county(year,variables)

['NAME', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_041E', 'B01001_042E', 'B01001_043E', 'B01001_001E', 'state', 'county']


In [97]:
df19['B01001_017E'] = df19['B01001_017E'].apply(changeInteger)
df19['B01001_018E'] = df19['B01001_018E'].apply(changeInteger)
df19['B01001_019E'] = df19['B01001_019E'].apply(changeInteger)
df19['B01001_041E'] = df19['B01001_041E'].apply(changeInteger)
df19['B01001_042E'] = df19['B01001_042E'].apply(changeInteger)
df19['B01001_043E'] = df19['B01001_043E'].apply(changeInteger)
df19['B01001_001E'] = df19['B01001_001E'].apply(changeInteger)

In [98]:
df19['percent_age_55_to_64'] = ((df19['B01001_017E']+df19['B01001_018E'] +df19['B01001_019E']+df19['B01001_041E']+df19['B01001_042E']+df19['B01001_043E'])/df19['B01001_001E'])*100

In [99]:
df19['fips'] = df19['state code'] + df19['county code']
df19.head()

,county_name,B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E,state code,county code,percent_age_55_to_64,fips
0,"Cumberland County, Maine",9656,2828,3884,9556,3038,4320,274712,23,005,12.115233,23005
1,"Franklin County, Maine",1013,381,608,877,478,578,29840,23,007,13.186997,23007
2,"Hancock County, Maine",2193,732,771,2292,633,986,53208,23,009,14.296722,23009
3,"Kennebec County, Maine",4286,1437,1836,4351,1607,1741,120559,23,011,12.656044,23011
4,"Knox County, Maine",1450,481,803,1584,425,794,40879,23,013,13.544852,23013


In [100]:
df19.shape

(1882, 12)

### Combine 19 dataframes together

In [101]:
df = df1[['fips','per_capita_income']].merge(df2[['fips', 'percent_below_poverty']], on = 'fips', how = 'left')

In [102]:
df = df.merge(df3[['fips', 'percent_no_highschool_diploma']], on = 'fips', how = 'left')
df = df.merge(df4[['fips', 'percent_mobile_homes']], on = 'fips', how = 'left')
df = df.merge(df5[['fips', 'percent_no_vehicle']], on = 'fips', how = 'left')
df = df.merge(df7[['fips', 'percent_overcrowding']], on = 'fips', how = 'left')
df = df.merge(df8[['fips', 'percent_multi_unit_housing']], on = 'fips', how = 'left')
df = df.merge(df9[['fips', 'percent_age_65_and_older']], on = 'fips', how = 'left')
df = df.merge(df10[['fips', 'percent_limited_english_abilities']], on = 'fips', how = 'left')
df = df.merge(df12[['fips', 'percent_minorities']], on = 'fips', how = 'left')
df = df.merge(df14[['fips', 'percent_institutionalized_in_group_quarters']], on = 'fips', how = 'left')
df = df.merge(df16[['fips', 'percent_age_18_to_34']], on = 'fips', how = 'left')
df = df.merge(df17[['fips', 'percent_age_35_to_44']], on = 'fips', how = 'left')
df = df.merge(df18[['fips', 'percent_age_45_to_54']], on = 'fips', how = 'left')
df = df.merge(df19[['fips', 'percent_age_55_to_64']], on = 'fips', how = 'left')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,27095,21236,10.778708,14.302092,7.163574,5.951190,NaN,6.998834,15.541087,3.168505,NaN,3.035790,23.230728,13.329738,13.976962,9.847055
1,27097,22348,9.238518,14.248729,9.169323,4.101111,1.200507,6.198400,15.268910,2.530148,NaN,1.774312,21.460584,13.190693,15.081655,10.240668
2,27099,23956,12.041345,14.703225,1.807582,6.293528,3.086065,7.215825,18.387790,8.551203,NaN,1.494729,19.924477,12.928096,13.887869,10.350344
3,27103,24917,11.323224,7.776017,7.807989,6.994754,1.840286,11.349377,11.711827,5.630485,NaN,5.036659,28.763150,12.645840,14.615875,9.859739
4,27105,21047,19.024611,21.855011,3.729317,9.356436,NaN,7.390084,16.462416,16.528748,NaN,3.145621,19.910692,12.016869,14.671297,10.905483


In [103]:
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,27095,21236,10.778708,14.302092,7.163574,5.951190,NaN,6.998834,15.541087,3.168505,NaN,3.035790,23.230728,13.329738,13.976962,9.847055
1,27097,22348,9.238518,14.248729,9.169323,4.101111,1.200507,6.198400,15.268910,2.530148,NaN,1.774312,21.460584,13.190693,15.081655,10.240668
2,27099,23956,12.041345,14.703225,1.807582,6.293528,3.086065,7.215825,18.387790,8.551203,NaN,1.494729,19.924477,12.928096,13.887869,10.350344
3,27103,24917,11.323224,7.776017,7.807989,6.994754,1.840286,11.349377,11.711827,5.630485,NaN,5.036659,28.763150,12.645840,14.615875,9.859739
4,27105,21047,19.024611,21.855011,3.729317,9.356436,NaN,7.390084,16.462416,16.528748,NaN,3.145621,19.910692,12.016869,14.671297,10.905483


In [104]:
df = df.sort_values('fips')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
585,01001,22270,10.686882,15.586557,22.569650,4.935705,1.411765,1.141599,10.858251,3.544326,NaN,NaN,21.559343,15.577100,14.634146,10.415349
586,01003,25673,11.412840,12.955327,16.298765,4.422221,1.721294,12.396625,16.438201,4.542152,14.453384,0.910877,21.156247,13.363915,14.189976,11.331912
587,01005,16184,23.179403,30.958128,28.070661,10.723477,1.677628,0.688791,12.968033,3.977725,NaN,10.023824,23.678128,14.984177,13.600967,11.549266
588,01007,18720,11.934156,31.193694,29.016506,5.058155,NaN,5.057686,12.112584,1.713745,NaN,4.415609,23.865758,16.542196,12.285288,11.043689
589,01009,19406,10.639650,26.329801,26.212837,3.882521,2.667582,0.630668,13.518986,5.975612,NaN,0.967690,22.262244,13.988495,14.212498,11.332724


In [105]:
df.to_csv('SDoH_3years/Social_Economic_3years_2007.csv')